In [4]:
# testing out that reading from .env works as expected
import os
from hdbcli import dbapi
from dotenv import load_dotenv

load_dotenv()
os.environ.get("TEST")

# Connection constants for space PYTHONCODESAMPLES
dbHost="9ee99a70-9d98-481d-ab94-00b0417b66c5.hana.prod-eu10.hanacloud.ondemand.com" #you should have noted them above
dbPort=443
dbUser="SP_PROJECTTEST#PYTHON"
dbPwd=os.environ.get("HANA_PWD_PYTHONCODESAMPLES")  #load pwd from .env file
dbIngestionSchema="SP_PROJECTTEST#PYTHON"
dbConsumptionSchema="SP_PROJECTTEST"
dbExampleTable="V_World_GDP_RAW"
dbExampleView="V_SalesOrder_CV"

In [9]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# creation of central connection objects. Set echo=True, if you need a log of all statements issued towards the DB
engine = create_engine(f'hana://{dbUser}:{dbPwd}@{dbHost}:{dbPort}', connect_args={
    'sslTrustStore' : "",
    'encrypt' : 'true',
    'sslHostNameInCertificate' : '*'
}, echo=True)

/var/folders/mg/phh0t0n96ndg9y12t2rshdm80000gn/T/ipykernel_45113/1632164404.py:5: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'sqlalchemy_hana.dialect.HANAHDBCLIDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(f'hana://{dbUser}:{dbPwd}@{dbHost}:{dbPort}', connect_args={


In [12]:
connection = engine.connect()
# Session = sessionmaker(bind=engine)
# connection = Session()

In [14]:
stmt = text(f'select top 2 * from "{dbConsumptionSchema}"."{dbExampleView}"')
result = connection.execute(stmt)
for row in result:
    print(row)

2023-03-23 13:39:35,680 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-23 13:39:35,682 INFO sqlalchemy.engine.Engine select top 2 * from "SP_PROJECTTEST"."V_SalesOrder_CV"
2023-03-23 13:39:35,682 INFO sqlalchemy.engine.Engine [dialect hana+hdbcli does not support caching 0.00236s] ()


/var/folders/mg/phh0t0n96ndg9y12t2rshdm80000gn/T/ipykernel_45113/890756207.py:2: SAWarning: Dialect hana:hdbcli will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  result = connection.execute(stmt)


('0500000111', '0000000010', 'RC-1054', None, 'USD', 'I', None, 'EA', datetime.date(2018, 8, 21), '0500000111', '0000000005', datetime.date(2018, 6, 12), '0000000005', datetime.date(2018, 6, 16), 'K4', '2018006', None, '0100000020', 'APJ', 'USD', 'C', 'C', 'C', Decimal('5499'), Decimal('4811.63'), Decimal('687.37'), Decimal('2'), Decimal('71749'), Decimal('62780.45'), Decimal('8968.37'))
('0500000111', '0000000020', 'MB-1033', None, 'USD', 'I', None, 'EA', datetime.date(2018, 8, 21), '0500000111', '0000000005', datetime.date(2018, 6, 12), '0000000005', datetime.date(2018, 6, 16), 'K4', '2018006', None, '0100000020', 'APJ', 'USD', 'C', 'C', 'C', Decimal('3999'), Decimal('3499.13'), Decimal('499.87'), Decimal('3'), Decimal('71749'), Decimal('62780.45'), Decimal('8968.37'))


In [15]:
engine.dialect.get_schema_names(connection=connection)

2023-03-23 13:39:48,169 INFO sqlalchemy.engine.Engine SELECT SCHEMA_NAME FROM SYS.SCHEMAS
2023-03-23 13:39:48,170 INFO sqlalchemy.engine.Engine [dialect hana+hdbcli does not support caching 0.00123s] ()


['dwc_global',
 'dwc_tenant_owner',
 'pal_stem_tfidf',
 'rob_sub',
 'sap_pa_apl',
 'sp_projecttest',
 'SP_PROJECTTEST#PYTHON',
 'sp_projecttest$tec',
 'sys',
 'system',
 '_sys_afl',
 '_sys_bi',
 '_sys_di',
 '_sys_plan_stability',
 '_sys_sql_analyzer',
 '_sys_task']

In [16]:
engine.dialect.get_view_names(connection=connection, schema=dbConsumptionSchema)

2023-03-23 13:39:55,689 INFO sqlalchemy.engine.Engine SELECT VIEW_NAME FROM SYS.VIEWS WHERE SCHEMA_NAME=?
2023-03-23 13:39:55,690 INFO sqlalchemy.engine.Engine [dialect hana+hdbcli does not support caching 0.00137s] ('SP_PROJECTTEST',)


['V_Product', 'V_SalesOrder_CV', 'V_World_GDP_RAW']

In [21]:
import pandas
# df = pandas.read_sql_table(stmt, con=engine)
# Doesn't work some bug in sql alchemy, would have been useful we switch back to hdicli